#PACKAGES LOAD

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
import warnings
warnings.filterwarnings("ignore")
print("packages loaded")
print("tf version is%s" % (tf.__version__))

packages loaded
tf version is1.14.0


In [2]:
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

In [3]:
# mnist
mnist = input_data.read_data_sets('data/', one_hot=True)
mnist
trainimg = mnist.train.images
trainlabel = mnist.train.labels
testimg = mnist.test.images
testlabel = mnist.test.labels
print("mnust loaded")

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
mnust loaded


In [4]:
print(mnist)
print(trainimg.shape)
print(trainlabel.shape)

Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x000001F23D840A08>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x000001F240813B88>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x000001F240813B48>)
(55000, 784)
(55000, 10)


In [5]:
# NETWORK TOPOLOGIES
n_hideen_1 = 256
n_hideen_2 = 128
n_input = 784
n_classes = 10
epochs = 50
batch_size = 100
display_step = 10
# INPUTS AND OUTPUTS
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [6]:
#NETWORK PARAMETERS
stddev = 0.1
weights = {
    'h1' : tf.Variable(tf.random_normal([n_input, n_hideen_1], stddev=stddev)),
    'h2' : tf.Variable(tf.random_normal([n_hideen_1, n_hideen_2], stddev=stddev)),
    'out': tf.Variable(tf.random_normal([n_hideen_2, n_classes], stddev=stddev))
}
biases = {
    'b1' : tf.Variable(tf.random_normal([n_hideen_1])),
    'b2' : tf.Variable(tf.random_normal([n_hideen_2])),
    'out' : tf.Variable(tf.random_normal([n_classes]))
}

In [7]:
#MLP AS FUNCTION
def multilayer_perceptron(_X, _weights, _biases) :
    layer1 = tf.nn.relu(tf.add(tf.matmul(_X, _weights['h1']), _biases['b1']))
    layer2 = tf.nn.relu(tf.add(tf.matmul(layer1, _weights['h2']), _biases['b2']))
    return (tf.matmul(layer2, _weights['out'])+ _biases['out'])

In [9]:
pred = multilayer_perceptron(x, weights, biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits= pred))
optm = tf.train.AdamOptimizer(learning_rate = 0.01).minimize(cost)
corr = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accr = tf.reduce_mean(tf.cast(corr, "float"))

In [10]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [12]:
#run
for epoch in range(epochs):
    avg_cost = 0.
    total_batch = int(mnist.train.num_examples/batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feeds = {x : batch_xs, y: batch_ys}
        sess.run(optm, feed_dict = feeds)
        avg_cost += sess.run(cost, feed_dict= feeds)
        avg_cost = avg_cost / total_batch
    #DIsplay
    if (epoch+1) % display_step == 0 :
        print("Epoch: %03d/%03d cost : %.0f" % (epoch, epochs, avg_cost))
        feeds = {x: batch_xs, y: batch_ys}
        train_acc = sess.run(accr, feed_dict = feeds)
        print("TRAIN ACCURACY: %.3f" % (train_acc))
        feeds = {x:mnist.test.images, y:mnist.test.labels}
        test_acc =sess.run(accr, feed_dict =feeds)
        print("TEST ACCURACY : %.3f" % (test_acc))

Epoch: 009/050 cost : 0
TRAIN ACCURACY: 0.980
TEST ACCURACY : 0.973
Epoch: 019/050 cost : 0
TRAIN ACCURACY: 0.980
TEST ACCURACY : 0.967
Epoch: 029/050 cost : 0
TRAIN ACCURACY: 0.990
TEST ACCURACY : 0.973
Epoch: 039/050 cost : 0
TRAIN ACCURACY: 1.000
TEST ACCURACY : 0.973
Epoch: 049/050 cost : 0
TRAIN ACCURACY: 0.990
TEST ACCURACY : 0.975


In [32]:
batch_xs.shape

(100, 784)

In [27]:
mnist.test.labels

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])